In [2]:
import pandas

In [77]:
import pandas as pd
from bs4 import BeautifulSoup

with open('data/nt_full_060124.html', 'r', encoding='utf-8') as f:

    html_content = f.read()

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract rows
    table_content = soup.select_one('tbody.table-content')
    # Extracting data for each player
data = []
for row in table_content.select('tr.hasCaptainsCol'):

    name = row.select_one('td.name-cell span').text
    position = row.select_one('td.position-cell ft-tooltip')['text']
    team = row.select_one('td.team-cell ft-tooltip')['text']
    price = row.select_one('td.price-cell').text
    ownership_pct = row.select_one('td.selectedBy-cell span.value').text
    captain_pct = row.select_one('td.captainedBy-cell span.value').text
    points = row.select_one('td.points-cell ft-bubble span').text
    
    data.append([name, position, team, price, ownership_pct, captain_pct, points])

    
# Convert data to a pandas DataFrame
norsk_tipping = pd.DataFrame(data)

print(norsk_tipping)


                    0         1                  2      3       4       5  \
0           C. Palmer  Midtbane            Chelsea   5.5M  38.55%   0.35%   
1            M. Salah  Midtbane          Liverpool  12.5M  62.38%  26.11%   
2            M. Olise  Midtbane     Crystal Palace   6.0M   0.72%   0.03%   
3           M. Kilman   Forsvar             Wolves   4.3M   0.87%       -   
4           C. Dawson   Forsvar             Wolves   4.2M   1.01%       -   
..                ...       ...                ...    ...     ...     ...   
530      L. Vigouroux    Keeper            Burnley   4.0M   0.03%       -   
531       R. Matthews    Keeper     Crystal Palace   4.0M   0.02%       -   
532  B. Baker-Boaitey  Midtbane           Brighton   4.5M       -       -   
533           S. Peck  Midtbane   Sheffield United   4.5M       -       -   
534       W. Kambwala   Forsvar  Manchester United   4.5M       -       -   

         6  
0    15.70  
1    15.10  
2    13.70  
3    12.30  
4    12.30

In [78]:
norsk_tipping.columns = ["Name", "Position", "Team", "Price", "Ownership", "Captain%", "PointsRound"]

In [79]:
norsk_tipping["Price"] = norsk_tipping["Price"].str.extract("^[+-]?([0-9]+([.][0-9]*)?|[.][0-9]+)M")[0]
norsk_tipping["Price"] = pd.to_numeric(norsk_tipping["Price"])

In [80]:
norsk_tipping.sort_values("Price", inplace=True)

In [81]:
norsk_tipping[["Initial", "Surname"]] = norsk_tipping["Name"].str.rsplit(". ", n=1, expand=True)

In [82]:
norsk_tipping

,Name,Position,Team,Price,Ownership,Captain%,PointsRound,Initial,Surname
267,H. Delcroix,Forsvar,Burnley,4.0,-,-,0.70,H,Delcroix
477,J. Shea,Keeper,Luton,4.0,2.92%,-,-,J,Shea
463,H. Ekdal,Forsvar,Burnley,4.0,0.47%,-,-,H,Ekdal
461,J. Amissah,Keeper,Sheffield United,4.0,0.17%,-,-,J,Amissah
459,A. A. Davies,Keeper,Sheffield United,4.0,0.21%,-,-,A. A,Davies
...,...,...,...,...,...,...,...,...,...
11,H. Son,Midtbane,Spurs,9.3,51.01%,8.22%,8.70,H,Son
301,K. De Bruyne,Midtbane,Manchester City,10.0,2.45%,0.47%,-,K,De Bruyne
16,B. Saka,Midtbane,Arsenal,10.2,39.27%,2.06%,7.70,B,Saka
1,M. Salah,Midtbane,Liverpool,12.5,62.38%,26.11%,15.10,M,Salah


## Get values from FPL Review

In [112]:
fplreview = pd.read_csv("confidential_data/fplreview_1704556496.csv")

point_columns = [ col for col in fplreview.columns if "_Pts" in col]

point_columns

fplreview["Total Points"] = fplreview[point_columns].sum(axis=1)

fplreview.sort_values("Total Points",ascending=False, inplace=True)
fplreview

,Pos,ID,Name,BV,SV,Team,21_xMins,21_Pts,22_xMins,22_Pts,...,29_xMins,29_Pts,30_xMins,30_Pts,31_xMins,31_Pts,32_xMins,32_Pts,Elite%,Total Points
354,F,355,Haaland,13.9,13.9,Man City,55,4.17,86,7.56,...,79,5.29,78,4.71,80,6.05,80,5.17,0.03,73.52
18,M,19,Saka,9.0,9.0,Arsenal,89,5.79,85,5.16,...,78,5.17,79,3.24,78,6.50,77,4.35,0.92,61.64
84,F,85,Solanke,7.1,6.9,Bournemouth,90,4.35,87,4.37,...,81,4.23,81,4.83,80,4.70,82,5.12,0.80,61.13
59,F,60,Watkins,8.9,8.6,Aston Villa,90,4.72,87,5.16,...,84,4.69,83,5.19,84,3.22,84,5.63,0.99,60.95
13,M,14,Ødegaard,8.4,8.4,Arsenal,87,5.22,85,4.88,...,81,4.94,81,3.17,78,5.89,79,4.17,0.04,58.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,D,305,Matip,4.9,4.9,Liverpool,0,0.00,0,0.00,...,0,0.00,0,0.00,0,0.00,0,0.00,0.00,0.00
298,M,299,Henderson,5.0,5.0,Liverpool,0,0.00,0,0.00,...,0,0.00,0,0.00,0,0.00,0,0.00,0.00,0.00
295,M,296,Fabinho,5.0,5.0,Liverpool,0,0.00,0,0.00,...,0,0.00,0,0.00,0,0.00,0,0.00,0.00,0.00
288,G,289,Adrián,3.9,3.9,Liverpool,0,0.00,0,0.00,...,0,0.00,0,0.00,0,0.00,0,0.00,0.00,0.00


In [85]:
norsk_tipping["Surname"].isin(fplreview["Name"].to_list()).value_counts()

Surname
True     406
False    129
Name: count, dtype: int64

In [90]:
df.columns

Index(['Name_nt', 'Position', 'Team', 'Price', 'Ownership', 'Captain%',
       'PointsRound', 'Initial', 'Surname', 'Pos', 'ID', 'Name_review', 'BV',
       'SV', '21_xMins', '21_Pts', '22_xMins', '22_Pts', '23_xMins', '23_Pts',
       '24_xMins', '24_Pts', '25_xMins', '25_Pts', '26_xMins', '26_Pts',
       '27_xMins', '27_Pts', '28_xMins', '28_Pts', '29_xMins', '29_Pts',
       '30_xMins', '30_Pts', '31_xMins', '31_Pts', '32_xMins', '32_Pts',
       'Elite%', 'Total Points'],
      dtype='object')

In [91]:
df[["Surname", "Name_review"]]

,Surname,Name_review
0,Delcroix,Delcroix
1,Shea,Shea
2,Ekdal,Ekdal
3,Amissah,NaN
4,Davies,NaN
...,...,...
530,Son,Son
531,De Bruyne,NaN
532,Saka,Saka
533,Salah,Salah


In [137]:
fplreview["Name"] = fplreview["Name"].str.rsplit(".", n=1, expand=True).fillna("").apply(lambda x : max(x,key=len),axis=1)

In [138]:
mapper = {"Man City": "Manchester City",
          "Man Utd": "Manchester United",
          "Nott'm Forest": "Nottingham Forest",
          "Sheffield Utd": "Sheffield United",
         }

In [139]:
fplreview["Team"] = fplreview.Team.replace(mapper)

In [140]:
df = norsk_tipping.merge(fplreview,left_on=["Team", "Surname"], right_on=["Team", "Name"], how="left",suffixes=("_nt", "_review"))

In [141]:
df

,Name_nt,Position,Team,Price,Ownership,Captain%,PointsRound,Initial,Surname,Pos,...,29_xMins,29_Pts,30_xMins,30_Pts,31_xMins,31_Pts,32_xMins,32_Pts,Elite%,Total Points
0,H. Delcroix,Forsvar,Burnley,4.0,-,-,0.70,H,Delcroix,D,...,25.0,0.91,26.0,0.54,26.0,0.90,28.0,0.72,0.00,7.66
1,J. Shea,Keeper,Luton,4.0,2.92%,-,-,J,Shea,G,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.00
2,H. Ekdal,Forsvar,Burnley,4.0,0.47%,-,-,H,Ekdal,D,...,25.0,0.80,28.0,0.48,28.0,0.87,28.0,0.62,0.00,6.34
3,J. Amissah,Keeper,Sheffield United,4.0,0.17%,-,-,J,Amissah,G,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.00
4,A. A. Davies,Keeper,Sheffield United,4.0,0.21%,-,-,A. A,Davies,M,...,20.0,0.75,20.0,0.79,20.0,0.71,20.0,0.77,0.00,8.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,H. Son,Midtbane,Spurs,9.3,51.01%,8.22%,8.70,H,Son,M,...,84.0,5.17,85.0,7.53,86.0,5.38,85.0,6.85,0.81,47.64
535,K. De Bruyne,Midtbane,Manchester City,10.0,2.45%,0.47%,-,K,De Bruyne,M,...,56.0,3.45,56.0,3.20,56.0,3.84,55.0,3.40,0.00,50.13
536,B. Saka,Midtbane,Arsenal,10.2,39.27%,2.06%,7.70,B,Saka,M,...,78.0,5.17,79.0,3.24,78.0,6.50,77.0,4.35,0.92,61.64
537,M. Salah,Midtbane,Liverpool,12.5,62.38%,26.11%,15.10,M,Salah,M,...,80.0,5.31,79.0,6.16,79.0,8.05,79.0,5.25,0.92,54.39


In [146]:
df[df.Pos.isna()].tail(50)

,Name_nt,Position,Team,Price,Ownership,Captain%,PointsRound,Initial,Surname,Pos,...,29_xMins,29_Pts,30_xMins,30_Pts,31_xMins,31_Pts,32_xMins,32_Pts,Elite%,Total Points
338,D. Alli,Midtbane,Everton,5.0,0.05%,-,-,D,Alli,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
345,V. Lindelöf,Forsvar,Manchester United,5.1,0.19%,-,-,V,Lindelöf,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349,M. Tresor,Midtbane,Burnley,5.2,-,-,0.70,M,Tresor,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
353,Danilo,Midtbane,Nottingham Forest,5.2,0.26%,-,2.30,Danilo,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,Beto,Angrep,Everton,5.2,-,-,1.00,Beto,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,B. Traore,Angrep,Sheffield United,5.2,0.82%,-,-,B,Traore,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,P. Porro,Forsvar,Spurs,5.2,38.79%,0.09%,2.30,P,Porro,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
362,B. De-Cordova Reid,Midtbane,Fulham,5.3,0.40%,-,7.30,B,De-Cordova Reid,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365,M. Kovacic,Midtbane,Manchester City,5.3,0.47%,-,2.30,M,Kovacic,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
366,F. Silva,Angrep,Wolves,5.3,0.63%,-,-,F,Silva,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
fplreview[fplreview.Team.isin(["Manchester City", "Liverpool"])].head(50)

,Pos,ID,Name,BV,SV,Team,21_xMins,21_Pts,22_xMins,22_Pts,...,29_xMins,29_Pts,30_xMins,30_Pts,31_xMins,31_Pts,32_xMins,32_Pts,Elite%,Total Points
354,F,355,Haaland,13.9,13.9,Manchester City,55,4.17,86,7.56,...,79,5.29,78,4.71,80,6.05,80,5.17,0.03,73.52
307,M,308,Salah,13.2,13.0,Liverpool,0,0.00,5,0.39,...,80,5.31,79,6.16,79,8.05,79,5.25,0.92,54.39
289,D,290,Alexander-Arnold,8.5,8.5,Liverpool,92,4.43,88,4.70,...,80,3.81,79,4.16,78,5.84,79,3.57,0.80,51.19
348,M,349,De Bruyne,10.2,10.2,Manchester City,65,4.37,63,4.97,...,56,3.45,56,3.20,56,3.84,55,3.40,0.00,50.13
351,G,352,Ederson M,5.5,5.5,Manchester City,93,3.75,92,4.39,...,89,3.48,89,3.59,89,3.75,89,3.93,0.07,48.85
352,M,353,Foden,7.8,7.8,Manchester City,72,4.22,66,4.59,...,65,3.51,63,3.20,63,3.79,63,3.53,0.25,48.27
342,F,343,Alvarez,7.0,7.0,Manchester City,72,4.55,62,4.48,...,58,3.32,59,3.10,60,3.80,61,3.41,0.72,47.86
292,F,293,Darwin,7.4,7.4,Liverpool,73,4.83,70,4.59,...,57,3.42,60,4.08,59,5.03,59,3.55,0.18,47.50
290,G,291,Becker,5.7,5.7,Liverpool,94,3.73,92,3.86,...,90,3.64,90,3.70,90,4.41,90,3.52,0.00,44.61
302,M,303,Luis Díaz,7.3,7.3,Liverpool,72,4.11,70,4.03,...,57,3.13,60,3.66,58,4.50,58,3.18,0.00,42.94
